In [ ]:
import time
import cv2
import numpy as np
import ExporterFileText

from Decode import jpeg_decode, ycbcr_to_rgb
from Encode import rgb_to_ycbcr, subsampling_422, jpeg_encode

start = time.time()
# Read image
img_path = 'old_street.jpg'
img = cv2.imread(img_path)
#img1 = cv2.resize(img, (400, 400), interpolation=cv2.INTER_LINEAR)
img1 = img
# Convert the image to YCbCr color spaces
width, height, _ = img1.shape
ycbcr_image = cv2.cvtColor(img1, cv2.COLOR_BGR2YCR_CB)
ycbcr_image = ycbcr_image
y, u, v = cv2.split(ycbcr_image)
# Subsample Cb and Cr components using 4:2:2 sampling
subsampled_y, subsampled_u, subsampled_v = subsampling_422(ycbcr_image)
# Encoding each channel
print("Encoding channel y")
huff_y, huff_table_y = jpeg_encode(subsampled_y, quality=50)
end1 = time.time()
print("Encoding channel Cb")
huff_u, huff_table_u = jpeg_encode(subsampled_u, quality=50)
end2 = time.time()
print("Encoding channel Cr")
huff_v, huff_table_v = jpeg_encode(subsampled_v, quality=50)


# v channel
print("huff v shape ", np.array(huff_v).shape)

np.savez_compressed('data_huff_v.npz', huff_v)

dict_data = np.load('data_huff_v.npz')
# extract the first array
data_huff_v = dict_data['arr_0']

print("data huff v shape ", np.array(data_huff_v).shape)

# u channel

np.save('data_huff_u_txt.txt', np.array(huff_u))
print("Saved")

print("huff u shape ", np.array(huff_u).shape)

np.savez_compressed('data_huff_u.npz', huff_u)

dict_data = np.load('data_huff_u.npz')
# extract the first array
data_huff_u = dict_data['arr_0']

print("data huff v shape ", np.array(data_huff_u).shape)

# y channel

print("huff u shape ", np.array(huff_y).shape)
huff_y_string = map(str, huff_y)
huff_y_flatten = np.array(huff_y)
huff_y_flatten.flatten()
huff_y_flatten_string = map(str, huff_y_flatten)
np.savez_compressed('data_huff_y.npz', huff_y)
np.savez_compressed('data_huff_y_string.npz', huff_y_string)
print(" Huff y flatten string shape: ", np.array( huff_y_flatten_string).shape)
np.savez_compressed('data_huff_y_flatten_string.npz', huff_y_flatten_string)

dict_data = np.load('data_huff_y.npz')
# extract the first array
data_huff_y = dict_data['arr_0']

print("data huff v shape ", np.array(data_huff_y).shape)



# huff_y_key = 'huff_y'
# huff_u_key = 'huff_u'
# huff_v_key = 'huff_v'
# huff_table_y_key = 'huff_table_y'
# huff_table_u_key = 'huff_table_u'
# huff_table_v_key = 'huff_table_v'

# np.savez_compressed('data_huff.npz', huff_y_key = huff_y, huff_u_key = huff_u, huff_v_key = huff_v, 
#                     huff_table_y_key = huff_table_y, huff_table_u_key = huff_table_u, huff_table_v_key = huff_table_v)

# dict_data = np.load('data_huff.npz')
# # extract the first array
# data_huff_v = dict_data[huff_v_key]

# print("huff v shape ", np.array(huff_v).shape)
# print("data huff v shape ", np.array(data_huff_v).shape)

huff_y_key = 'huff_y'
huff_u_key = 'huff_u'
huff_v_key = 'huff_v'
huff_table_y_key = 'huff_table_y'
huff_table_u_key = 'huff_table_u'
huff_table_v_key = 'huff_table_v'

data_args = {}
data_args[huff_y_key] = huff_y
data_args[huff_u_key] = huff_u
data_args[huff_v_key] = huff_v
data_args[huff_table_y_key] = huff_table_y
data_args[huff_table_u_key] = huff_table_u
data_args[huff_table_v_key] = huff_table_v

ExporterFileText.ExportToText(data_args)





end3 = time.time()
# Decoding
print("Decoding channel y")
y_decoded = jpeg_decode(huff_y, huff_table_y, 50)
end4 = time.time()
print("Decoding channel Cb")
u_decoded = jpeg_decode(huff_u, huff_table_u, 50)
end5 = time.time()
print("Decoding channel Cr")
v_decoded = jpeg_decode(huff_v, huff_table_v, 50)
end6 = time.time()
# Show result
h, w = y_decoded.shape
u_decoded = cv2.resize(u_decoded, (w, h), interpolation=cv2.INTER_LINEAR)
v_decoded = cv2.resize(v_decoded, (w, h), interpolation=cv2.INTER_LINEAR)
img_decoded = cv2.merge([y_decoded, u_decoded, v_decoded])
img_decoded = img_decoded
img_decoded = img_decoded.astype(np.uint8)
# Convert image to RGB
img_decoded_rgb = cv2.cvtColor(img_decoded, cv2.COLOR_YCrCb2BGR) #ycbcr_to_rgb(img_decoded)
print(end1 - start)
print(end4 - start)
cv2.imshow('input', img1)
cv2.imshow('JPEG img', img_decoded_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()


Encoding channel y
Encoding channel Cb
Encoding channel Cr
huff v shape  (210, 140)
data huff v shape  (210, 140)
Saved
huff u shape  (210, 140)
data huff v shape  (210, 140)
huff u shape  (420, 280)
 Huff y flatten string shape:  ()
data huff v shape  (420, 280)
Decoding channel y
420 280
Decoding channel Cb
210 140
Decoding channel Cr
210 140
22.665807723999023
72.6139132976532


In [3]:
# u channel

np.savez('data_huff_u_txt.npz', huff_u)
print("Saved")

Saved


In [ ]:
print(huff_v.shape)